In [100]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from utilities.data_preprocessors import get_random_data, MetricsAfterEachEpoch
from keras import regularizers

In [116]:
global MAX_EPOCHS, MAX_BATCH_SIZE, architecture, label, keywords
MAX_EPOCHS, MAX_BATCH_SIZE = 15, 1024
architecture = 'LeNet5'
label, keywords = 'noisy_advanced-experiments_test', 'advanced_testing'

In [117]:
MAX_TRAINING = 40000
MAX_VALIDATION = 5000
MAX_TESTING = 5000

base = 'noisy_dataset/noisy/cifar_10_{}.npy'
x_train_max, y_train = np.load(base.format('x_train')), np.load(base.format('y_train'))
x_test_max, y_test = np.load(base.format('x_test')), np.load(base.format('y_test'))
# cifar = tf.keras.datasets.cifar10 
# (_, _), (x_test_max, y_test) = cifar.load_data()
y_train_max, y_test_max = to_categorical(y_train, num_classes=10), to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*4)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_val, y_val = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], int(MAX_TESTING/2))
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=40000, Y=40000
Number of Validation Samples: X=2500, Y=2500
Number of Test Samples: X=5000, Y=5000


In [118]:
def model_extractor(activation_func, weight_decay=1e-4):
    # Creating a LeNet5 Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))
    model.add(BatchNormalization())

    # #Instantiating Layer 2
    model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))
    model.add(BatchNormalization())

    # #Instantiating Layer 3
    model.add(Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Flatten())

    #Instantiating Layer 4
    model.add(Dense(84, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [119]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=MAX_EPOCHS):
    model = model_extractor(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
    
    #Saving the model
    if label == 'RAdam':
        model.save("../../saved_models_advanced/{}_{}_{}.hdf5".format('RAdam', activation_func, architecture))
    else: 
        model.save("../../saved_models_advanced/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, architecture))
    return score, history

In [120]:
testing_optimizers = ['RAdam'] #['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh', 'relu', 'selu'] #'selu'


for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.05), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)


        bg = (score_returned[0], score_returned[1], score_returned[2], score_returned[3], history.get('loss'),\
          history.get('accuracy'), history.get('mae'), history.get('mae'))

        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, architecture, label, keywords)
        writeToDB(optimizer, activation, architecture, bg, label, keywords)

print ("Completed Execution for architecture={}".format(architecture))

Train on 40000 samples
Epoch 1/15
40000/40000 [==============================] - 9s 220us/sample - loss: 1.5703 - acc: 0.4081 - mean_absolute_error: 0.1409 - mean_squared_error: 0.0722
Epoch 2/15
40000/40000 [==============================] - 5s 129us/sample - loss: 1.1952 - acc: 0.5192 - mean_absolute_error: 0.1213 - mean_squared_error: 0.0608
Epoch 3/15
40000/40000 [==============================] - 5s 126us/sample - loss: 1.1478 - acc: 0.5448 - mean_absolute_error: 0.1161 - mean_squared_error: 0.0586
Epoch 4/15
40000/40000 [==============================] - 5s 122us/sample - loss: 1.0908 - acc: 0.5680 - mean_absolute_error: 0.1109 - mean_squared_error: 0.0559
Epoch 5/15
40000/40000 [==============================] - 5s 122us/sample - loss: 1.0888 - acc: 0.5711 - mean_absolute_error: 0.1098 - mean_squared_error: 0.0556
Epoch 6/15
40000/40000 [==============================] - 5s 122us/sample - loss: 1.0792 - acc: 0.5753 - mean_absolute_error: 0.1086 - mean_squared_error: 0.0550
Epoch

In [77]:
# 10%
# def model_extractor(activation_func):
#     weight_decay = 1e-4
#     # Creating a LeNet5 Classifier
#     model = Sequential()

#     #Instantiating Layer 1
#     model.add(Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
#                      kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))
#     model.add(BatchNormalization())

#     # #Instantiating Layer 2
#     model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
#                     kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))
#     model.add(BatchNormalization())

#     # #Instantiating Layer 3
#     model.add(Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
#                     kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(Flatten())

#     #Instantiating Layer 4
#     model.add(Dense(84, activation=activation_func)) 

#     #Output Layer
#     model.add(Dense(10, activation='softmax'))
#     return model

22.88%